In [1]:
import cv2
import numpy as np
from collections import deque

In [39]:
buffer_size = 16
pts = deque(maxlen=buffer_size)

In [12]:
# mavi renk aralığı HSV
blueLower = (84, 98, 0)
blueUpper = (179, 255, 255)

In [52]:
video = cv2.VideoCapture(0)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
writer = cv2.VideoWriter(filename="images/colortracking1.mp4",
                         fourcc=cv2.VideoWriter_fourcc(*"DIVX"),
                         fps=50,
                         frameSize=(width, height))

while True:
    retval, img = video.read()
    if not retval: break
    img = cv2.flip(src=img, flipCode=1)
    
    # blur
    blurred = cv2.GaussianBlur(src=img, ksize=(11, 11), sigmaX=0)
    
    # hsv
    hsv = cv2.cvtColor(src=blurred, code=cv2.COLOR_BGR2HSV)
    
    # mask for blue
    mask = cv2.inRange(src=hsv, lowerb=blueLower, upperb=blueUpper)
    #cv2.imshow(winname="Mask", mat=mask)
    
    # maskenin etrafında kalan gürültüleri sil
    mask = cv2.erode(src=mask, kernel=None, iterations=2)
    mask = cv2.dilate(src=mask, kernel=None, iterations=2)
    #cv2.imshow(winname="Mask + Erode + Dilate", mat=mask)
    
    
    # kontur
    contours, hierarchy = cv2.findContours(image=mask.copy(), 
                                         mode=cv2.RETR_EXTERNAL, 
                                         method=cv2.CHAIN_APPROX_SIMPLE)
    
    center = None # nesnemizin merkezi olacak.
    
    if len(contours) > 0:
        
        # en büyük konturu alalım.
        c = max(contours, key=cv2.contourArea)
        
        # dikdörtgene çevirelim.
        rect = cv2.minAreaRect(points=c)
        
        ((x, y), (width, height), rotation) = rect
        
        s = f"""x:{int(x)}, y:{int(y)}, width:{int(width)}, height:{int(height)}, rotation:{int(rotation)}"""
        
        # kutucuk
        box = np.int64(cv2.boxPoints(rect))
        
        # moment
        M = cv2.moments(array=c)
        center = (int(M["m10"]/M["m00"]), int(M["m01"]/M["m00"]))
        
        # konturu çizdirelim.
        cv2.drawContours(image=img, contours=[box], contourIdx=0, color=(0, 255, 255), thickness=2)
        
        # merkeze bir tane nokta çizelim.
        cv2.circle(img=img, center=center, radius=5, color=(0, 255, 255), thickness=-1)
        
        # bilgileri ekrana yazdıralım.
        cv2.putText(img=img,
                    text=s,
                    org=(30, 30),
                    fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                    fontScale=1,
                    color=(255, 255, 255),
                    thickness=2)
        
        # deque
        pts.appendleft(center)
        
        for i in range(1, len(pts)):
            if pts[i-1] is None or pts[i] is None: continue
            cv2.line(img=img, pt1=pts[i-1], pt2=pts[i], color=(0, 255, 0), thickness=3)
        
    writer.write(image=img)
    cv2.imshow(winname="hsv image", mat=img)
    if cv2.waitKey(1) == ord("q"): break
    
writer.release()
video.release()
cv2.destroyAllWindows()

In [46]:
?np.round

Signature: np.round(a, decimals=0, out=None)
Docstring:
Round an array to the given number of decimals.

See Also
--------
around : equivalent function; see for details.
File:      c:\users\murat\anaconda3\lib\site-packages\numpy\core\fromnumeric.py
Type:      function
